# 使用ChArUco Board做相機校正


## 先取得校正要用到的點

In [35]:
import cv2
import cv2.aruco as aruco

cap = cv2.VideoCapture('CharUco_board.mp4') # 取得攝影機
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams  = aruco.DetectorParameters_create()  # 建立Aruco參數(detectMarker process會用到)
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX  #用subpixel的方式來收細角點

arucoDict    = aruco.Dictionary_get(aruco.DICT_6X6_250)  #我們用的是這個參數產生的ChArUco board

# 必須描述ChArUco board的尺寸規格
gridX        = 5 # 水平方向5格
gridY        = 7 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
# ArUco marker為2cmX2cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)  # 建立ChArUco board

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH))) # 取得攝影機的長寬
refinedStrategy = True  #
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []
while True:
    ret, frame = cap.read()  # 讀取一幀影像
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))  # 縮小畫面
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)  #偵測Aruco marker的角點位置(每個marker的四個角點), ids是每個marker的id, rejected是有錯或信心不足的marker的corner(也許是不符呵我們設定的DICT_6X6_250)

    #corners是17個 4*2的陣列(代表每個marker的四個角點)
    #ids是17個 1*1的陣列(代表每個marker的id) 每個marker的id是唯一的
    #rejected是多個 4*2的陣列(代表每個rejected的四個角點)

    #refinedStrategy代表是否用板子資訊去強化arucro的偵測
    #這個一定要在detectMarker之後再做
    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)

    #每100個frame存一次之後校正要用到的點對應關係
    if frameId % 100 == 50 and len(ids)==17: # 17 ArUco markers 如果全部正確被讀到才起來
        collectCorners.append(corners)
        collectIds.append(ids.ravel())
        collectFrames.append(frame)

    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)  #把marker的角點畫出來

    cv2.imshow('find points for calibration',frame)
    if cv2.waitKey(20) != -1:
        break

    frameId += 1

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


## 開始校正

In [36]:
import numpy as np
#把所有的校正要用到的點重新縮放後放到一個陣列中
#reshape中的-1代表讓電腦判斷那個維度
#collectCorners原本是好幾組的 17*(1*4*2)的陣列
#把它轉成好幾組的(4*2)的陣列
caliCorners=np.concatenate([np.array(x).reshape(-1,4,2) for x in collectCorners],axis=0)
counter=np.array([len(x) for x in collectIds]) #計算每筆DATA收集到多少點
caliIds=np.array(collectIds).ravel()  #把collectCorners所對應的ID排成一排，讓他和caliCorners一致
#初始化內部參數
cameraMatrixInit = np.array([[ 1000.,    0., frameWidth/2.],[    0., 1000., frameHeight/2.],[    0.,    0.,           1.]])
#初始化畸形變參數
distCoeffsInit   = np.zeros((5,1))
#用charucoBoard及收集到的點還來做校正
#給他收集到點的Aruco的角點、這些Aruco對應的ID、及ChArUco board板子布局
#他就會回傳內部參數(cameraMatrix),外部參數(rvects和tvects),及失真參數(distCoeffs)
ret, aruco_cameraMatrix, aruco_distCoeffs, aruco_rvects, aruco_tvects = aruco.calibrateCameraAruco(caliCorners,caliIds,counter,charucoBoard,(frameWidth,frameHeight),cameraMatrixInit,distCoeffsInit)
print(aruco_cameraMatrix)
print(aruco_distCoeffs)
print(aruco_rvects)
print(aruco_tvects)

[[913.01878047   0.         479.68086947]
 [  0.         920.3179607  293.38804426]
 [  0.           0.           1.        ]]
[[ 0.0678466 ]
 [-0.11907327]
 [-0.0044268 ]
 [-0.00262406]
 [ 0.00265484]]
(array([[ 1.85578111],
       [ 2.13572257],
       [-0.43417398]]), array([[ 2.02390891],
       [ 2.16242677],
       [-0.37060299]]), array([[ 1.95372558],
       [ 2.13947932],
       [-0.43930413]]), array([[ 2.07463475],
       [ 2.14272052],
       [-0.22373451]]), array([[ 1.84966262],
       [ 2.40866435],
       [-0.12727971]]), array([[ 2.01004494],
       [ 2.25552498],
       [-0.31535423]]), array([[ 2.03671404],
       [ 2.15606694],
       [-0.52940489]]), array([[ 2.12747067],
       [ 2.19250079],
       [-0.09167031]]), array([[ 2.22641555],
       [ 2.13736037],
       [-0.11893928]]), array([[ 2.23526767],
       [ 2.0336658 ],
       [-0.02519255]]), array([[ 2.23620562],
       [ 2.0917037 ],
       [-0.14035539]]))
(array([[-12.15349024],
       [-10.66690443],
 

In [ ]:
caliCorners=[]
caliIds    =[]
#把每一個CharucoBoard當作一個整體來做校正
for corners, ids, frame in zip(collectCorners,collectIds,collectFrames):
    #
    ret, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(corners,ids,frame,charucoBoard,aruco_cameraMatrix,aruco_distCoeffs)
    caliCorners.append(charucoCorners) #存板子內所有CORNER
    caliIds.append(charucoIds)  #存板子內所有ARUCO ID的排列

#用CharucoBoard校正
#一樣可以得到內部參數(cameraMatrix),外部參數(rvects和tvects),及失真參數(distCoeffs)
ret, charuco_cameraMatrix, charuco_distCoeffs, charuco_rvects, charuco_tvects = aruco.calibrateCameraCharuco(caliCorners,caliIds,charucoBoard,(frameWidth,frameHeight), aruco_cameraMatrix,aruco_distCoeffs)
print(charuco_cameraMatrix)
print(charuco_distCoeffs)
print(charuco_rvects)
print(charuco_tvects)

# 用校正好的相機找到每個CharUco_board的位置(3D位置)

In [ ]:
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
cap = cv2.VideoCapture('CharUco_board.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None and len(ids)>0:
        aruco.drawDetectedMarkers(frame, corners, ids)
        ret, rvect, tvect = aruco.estimatePoseBoard(corners, ids, charucoBoard, charuco_cameraMatrix, charuco_distCoeffs, None, None)
        if ret:
            aruco.drawAxis(frame, charuco_cameraMatrix, charuco_distCoeffs, rvect, tvect, squareSize)

    cv2.imshow('Estimation of the pose of a CharUco board with intrinsic camera parameters by charuco',frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:
#原始畫面有點大，為了有利於顯示這份講義所以縮小。
cap = cv2.VideoCapture('CharUco_board.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None and len(ids)>0:
        aruco.drawDetectedMarkers(frame, corners, ids)
        ret, rvect, tvect = aruco.estimatePoseBoard(corners, ids, charucoBoard, aruco_cameraMatrix, aruco_distCoeffs, None, None)
        if ret:
            aruco.drawAxis(frame, aruco_cameraMatrix, aruco_distCoeffs, rvect, tvect, squareSize)

    cv2.imshow('Estimation of the pose of a CharUco board with intrinsic camera parameters by aruco',frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

# 用校正好的相機找到每個Aruco的位置

In [ ]:
markerSize  = 6 #6cm
cap = cv2.VideoCapture('ArUco_marker.mp4')
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

hw2_output_video = cv2.VideoWriter('myHW2Video.mp4', cv2.VideoWriter_fourcc(*'XVID'),
                                   cap.get(cv2.CAP_PROP_FPS),
                                   (frameWidth,frameHeight))  #輸出的影片

nggyu_cap = cv2.VideoCapture('nggyu.mp4')
hyyyy_cap = cv2.VideoCapture('hyyyy.mp4')
esg_cap   = cv2.VideoCapture('esg.mp4')
ncat_cap = cv2.VideoCapture('ncat.mp4')
game_cap = cv2.VideoCapture('game.mp4')
vibe_cap = cv2.VideoCapture('vibe.mp4')

has_started = [False for _ in range(6)] #紀錄哪些id開始被偵測到了
while True:
    ret, frame = cap.read()
    _, nggyu_frame = nggyu_cap.read()
    _, hyyyy_frame = hyyyy_cap.read()
    _, esg_frame = esg_cap.read()
    _, ncat_frame = ncat_cap.read()
    _, game_frame = game_cap.read()
    _, vibe_frame = vibe_cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    frame = cv2.undistort(frame, charuco_cameraMatrix, charuco_distCoeffs)

    # #把nggyu影片貼合到一張新影像的左上角
    # nggyu_frame_full = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]),dtype=np.uint8)
    # nggyu_frame_full[:360,:480]=nggyu_frame[:360,90:90+480]
    #
    # #把hyyyy影片貼合到一張新影像的左上角

    #這些影片有黑邊，所以把他們切掉
    nggyu_frame_crop = np.zeros((nggyu_frame.shape[0], 460, nggyu_frame.shape[2]),dtype=np.uint8)
    nggyu_frame_crop[:,:,:]=nggyu_frame[:,90:550,:]

    ncat_frame_crop = np.zeros((ncat_frame.shape[0], 550, ncat_frame.shape[2]),dtype=np.uint8)
    ncat_frame_crop[:,:,:]=ncat_frame[:,85:635,:]

    esg_frame_crop = np.zeros((360, esg_frame.shape[1], esg_frame.shape[2]),dtype=np.uint8)
    esg_frame_crop[:,:,:]=esg_frame[60:420,:,:]



    #print(nggyu_frame_crop.shape)

    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if len(corners) > 0:
        #aruco.drawDetectedMarkers(frame, corners, ids)
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        for corner,id in zip(corners,ids):
            if id == 1:
                mask_image = np.ones_like(vibe_frame)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,0.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(vibe_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 2:
                mask_image = np.ones_like(esg_frame_crop)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,00.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(esg_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 3:
                mask_image = np.ones_like(nggyu_frame_crop)
                source_pts = np.array([[460,360.0],[0.0,360.0],[0.0,0.0],[460.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(nggyu_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 4:
                mask_image = np.ones_like(hyyyy_frame)
                source_pts = np.array([[480.0,360.0],[0.0,360.0],[0.0,0.0],[480.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(hyyyy_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 5:
                mask_image = np.ones_like(ncat_frame_crop)
                source_pts = np.array([[550.0,480.0],[0.0,480.0],[0.0,0.0],[550.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(ncat_frame_crop, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 6:
                mask_image = np.ones_like(game_frame)
                source_pts = np.array([[854.0,480.0],[00.0,480.0],[0.0,0.0],[854.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(game_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]

    #aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('Estimation of the pose of arUco marker with intrinsic camera parameters',frame)
    hw2_output_video.write(frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()
hw2_output_video.release()
nggyu_cap.release()
hyyyy_cap.release()
esg_cap.release()
ncat_cap.release()
game_cap.release()
vibe_cap.release()


In [40]:
markerSize  = 6 #6cm
cap = cv2.VideoCapture('ArUco_marker.mp4')
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

hw2_output_video = cv2.VideoWriter('myHW2Video.mp4', cv2.VideoWriter_fourcc(*'XVID'),
                                   cap.get(cv2.CAP_PROP_FPS),
                                   (frameWidth,frameHeight))  #輸出的影片

nggyu_cap = cv2.VideoCapture('nggyu.mp4')
hyyyy_cap = cv2.VideoCapture('hyyyy.mp4')
esg_cap   = cv2.VideoCapture('esg.mp4')
ncat_cap = cv2.VideoCapture('ncat.mp4')
game_cap = cv2.VideoCapture('game.mp4')
vibe_cap = cv2.VideoCapture('vibe.mp4')

has_started = [False for _ in range(6)] #紀錄哪些id開始被偵測到了
while True:
    ret, frame = cap.read()
    _, nggyu_frame = nggyu_cap.read()
    _, hyyyy_frame = hyyyy_cap.read()
    _, esg_frame = esg_cap.read()
    _, ncat_frame = ncat_cap.read()
    _, game_frame = game_cap.read()
    _, vibe_frame = vibe_cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    frame = cv2.undistort(frame, charuco_cameraMatrix, charuco_distCoeffs)

    # #把nggyu影片貼合到一張新影像的左上角
    # nggyu_frame_full = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]),dtype=np.uint8)
    # nggyu_frame_full[:360,:480]=nggyu_frame[:360,90:90+480]
    #
    # #把hyyyy影片貼合到一張新影像的左上角

    #這些影片有黑邊，所以把他們切掉
    nggyu_frame_crop = np.zeros((nggyu_frame.shape[0], 460, nggyu_frame.shape[2]),dtype=np.uint8)
    nggyu_frame_crop[:,:,:]=nggyu_frame[:,90:550,:]

    ncat_frame_crop = np.zeros((ncat_frame.shape[0], 550, ncat_frame.shape[2]),dtype=np.uint8)
    ncat_frame_crop[:,:,:]=ncat_frame[:,85:635,:]

    esg_frame_crop = np.zeros((360, esg_frame.shape[1], esg_frame.shape[2]),dtype=np.uint8)
    esg_frame_crop[:,:,:]=esg_frame[60:420,:,:]



    #print(nggyu_frame_crop.shape)

    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if len(corners) > 0:
        #aruco.drawDetectedMarkers(frame, corners, ids)
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        for corner,id in zip(corners,ids):
            if id == 1:
                mask_image = np.ones_like(vibe_frame)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,0.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(vibe_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 2:
                mask_image = np.ones_like(esg_frame_crop)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,00.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(esg_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 3:
                mask_image = np.ones_like(nggyu_frame_crop)
                source_pts = np.array([[460,360.0],[0.0,360.0],[0.0,0.0],[460.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(nggyu_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 4:
                mask_image = np.ones_like(hyyyy_frame)
                source_pts = np.array([[480.0,360.0],[0.0,360.0],[0.0,0.0],[480.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(hyyyy_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 5:
                mask_image = np.ones_like(ncat_frame_crop)
                source_pts = np.array([[550.0,480.0],[0.0,480.0],[0.0,0.0],[550.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(ncat_frame_crop, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 6:
                mask_image = np.ones_like(game_frame)
                source_pts = np.array([[854.0,480.0],[00.0,480.0],[0.0,0.0],[854.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(game_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]

    #aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('Estimation of the pose of arUco marker with intrinsic camera parameters',frame)
    hw2_output_video.write(frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()
hw2_output_video.release()
nggyu_cap.release()
hyyyy_cap.release()
esg_cap.release()
ncat_cap.release()
game_cap.release()
vibe_cap.release()


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [1]:
markerSize  = 6 #6cm
cap = cv2.VideoCapture('ArUco_marker.mp4')
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)
totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

hw2_output_video = cv2.VideoWriter('myHW2Video.mp4', cv2.VideoWriter_fourcc(*'XVID'),
                                   cap.get(cv2.CAP_PROP_FPS),
                                   (frameWidth,frameHeight))  #輸出的影片

nggyu_cap = cv2.VideoCapture('nggyu.mp4')
hyyyy_cap = cv2.VideoCapture('hyyyy.mp4')
esg_cap   = cv2.VideoCapture('esg.mp4')
ncat_cap = cv2.VideoCapture('ncat.mp4')
game_cap = cv2.VideoCapture('game.mp4')
vibe_cap = cv2.VideoCapture('vibe.mp4')

has_started = [False for _ in range(6)] #紀錄哪些id開始被偵測到了
while True:
    ret, frame = cap.read()
    _, nggyu_frame = nggyu_cap.read()
    _, hyyyy_frame = hyyyy_cap.read()
    _, esg_frame = esg_cap.read()
    _, ncat_frame = ncat_cap.read()
    _, game_frame = game_cap.read()
    _, vibe_frame = vibe_cap.read()
    if not ret:
        break

    frame = cv2.resize(frame,(frameWidth,frameHeight))
    frame = cv2.undistort(frame, charuco_cameraMatrix, charuco_distCoeffs)

    # #把nggyu影片貼合到一張新影像的左上角
    # nggyu_frame_full = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2]),dtype=np.uint8)
    # nggyu_frame_full[:360,:480]=nggyu_frame[:360,90:90+480]
    #
    # #把hyyyy影片貼合到一張新影像的左上角

    #這些影片有黑邊，所以把他們切掉
    nggyu_frame_crop = np.zeros((nggyu_frame.shape[0], 460, nggyu_frame.shape[2]),dtype=np.uint8)
    nggyu_frame_crop[:,:,:]=nggyu_frame[:,90:550,:]

    ncat_frame_crop = np.zeros((ncat_frame.shape[0], 550, ncat_frame.shape[2]),dtype=np.uint8)
    ncat_frame_crop[:,:,:]=ncat_frame[:,85:635,:]

    esg_frame_crop = np.zeros((360, esg_frame.shape[1], esg_frame.shape[2]),dtype=np.uint8)
    esg_frame_crop[:,:,:]=esg_frame[60:420,:,:]



    #print(nggyu_frame_crop.shape)

    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if len(corners) > 0:
        #aruco.drawDetectedMarkers(frame, corners, ids)
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        for corner,id in zip(corners,ids):
            if id == 1:
                mask_image = np.ones_like(vibe_frame)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,0.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(vibe_frame, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 2:
                mask_image = np.ones_like(esg_frame_crop)
                source_pts = np.array([[640.0,360.0],[0.0,360.0],[0.0,00.0],[640.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(esg_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 3:
                mask_image = np.ones_like(nggyu_frame_crop)
                source_pts = np.array([[460,360.0],[0.0,360.0],[0.0,0.0],[460.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(nggyu_frame_crop, M, (frameWidth, frameHeight), flags=cv2.INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 4:
                mask_image = np.ones_like(hyyyy_frame)
                source_pts = np.array([[480.0,360.0],[0.0,360.0],[0.0,0.0],[480.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(hyyyy_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 5:
                mask_image = np.ones_like(ncat_frame_crop)
                source_pts = np.array([[550.0,480.0],[0.0,480.0],[0.0,0.0],[550.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(ncat_frame_crop, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]
            if id == 6:
                mask_image = np.ones_like(game_frame)
                source_pts = np.array([[854.0,480.0],[00.0,480.0],[0.0,0.0],[854.0,0.0]])
                dest_pts = np.array(corner)
                M, mask = cv2.findHomography(source_pts,dest_pts, cv2.RANSAC,5.0)
                warped = cv2.warpPerspective(game_frame, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                mask_warped = cv2.warpPerspective(mask_image, M, (frameWidth, frameHeight), flags=cv2.  INTER_CUBIC)
                frame[np.where(mask_warped>0)] = warped[np.where(mask_warped>0)]

    #aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('Estimation of the pose of arUco marker with intrinsic camera parameters',frame)
    hw2_output_video.write(frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()
hw2_output_video.release()
nggyu_cap.release()
hyyyy_cap.release()
esg_cap.release()
ncat_cap.release()
game_cap.release()
vibe_cap.release()


NameError: name 'cv2' is not defined